

# MVP de Engenharia e Análise de Dados: Impacto das Redes Sociais na Saúde Mental

**Aluno:** Eric Koji Nakirimoto

## 1. Visão Geral do Projeto
Este projeto tem como objetivo construir um pipeline de dados completo (End-to-End) na nuvem utilizando a plataforma **Databricks free**. O foco central é analisar a correlação entre hábitos digitais — especificamente o uso de redes sociais e tempo de tela — e indicadores de saúde mental e bem-estar, como níveis de estresse, qualidade do sono e felicidade.

A análise busca responder se o estilo de vida digital moderno está impactando negativamente a saúde emocional dos usuários, fornecendo insights baseados em dados para uma discussão mais fundamentada sobre o tema.

## 2. Perguntas de Negócio (Insights)
Para guiar a modelagem e a análise dos dados, foram definidas as seguintes questões norteadoras:
1.  **Redes Sociais x Estresse:** Existe uma correlação entre o tempo gasto em redes sociais e o aumento do nível de estresse?
2.  **Redes Sociais x Felicidade:** O uso intensivo de plataformas digitais afeta o índice de felicidade reportado?
3.  **Redes Sociais x Sono:** O tempo de tela diário impacta a qualidade do sono?
4.  **Hábitos Saudáveis:** Qual a relação entre a frequência de exercícios físicos, qualidade do sono e a felicidade geral?

## 3. Fonte de Dados
Os dados utilizados neste projeto são públicos e foram obtidos através do Kaggle. Meu modelo é uma flat table pois os dados vieram nesse formato.
* **Dataset:** Social Media and Mental Health Balance
* **Origem:** [Kaggle - Ayesha Imran](https://www.kaggle.com/datasets/ayeshaimran123/social-media-and-mental-health-balance/data)
* **Conteúdo:** O conjunto de dados inclui informações sobre tempo de tela, frequência de exercícios, métricas de sono, níveis de estresse e índices de felicidade de diversos usuários.

## 4. Metodologia e Arquitetura
O projeto segue a arquitetura de referência **Medallion (Bronze, Silver, Gold)**, implementada via **PySpark** e **Spark SQL**:

* **Camada Bronze (Raw):** Ingestão dos dados brutos (arquivo CSV) para o Data Lake sem transformações, garantindo a preservação do histórico original.
* **Camada Silver (Trusted):** Limpeza, padronização de tipos de dados, renomeação de colunas (tradução e formatação), tratamento de valores nulos e remoção de duplicatas.
* **Camada Gold (Refined):** Agregação dos dados em visões analíticas focadas nas perguntas de negócio, prontas para consumo por ferramentas de visualização.



## 1. Etapa
Para essa etapa vou conectar o arquivo importado e converter em um dataframe para a limpeza e organização. Esse é uma base de dados de nível 10n segundo o kaggle, logo é sabido que ela poderá estar limpa e sem dados faltantes. Por fim, para fins didáticos serão feitas todas as etapas de limpeza e organização.

In [0]:
# 1. Leitura da tabela bruta (Bronze)
table_name = "workspace.default.mental_health_and_social_media_balance_dataset"
df_bronze = spark.read.table(table_name)

# 2. Visualizar os dados e o Schema
display(df_bronze)
df_bronze.printSchema()

## 2. Etapa
Para essa etapa, identificaram-se os tipos de dados, se há dados nulos os "not a number", contagem dos dados, verificação da integridade e algumas estatísticas descritivas

In [0]:
from pyspark.sql.functions import col, count, when, isnan, desc
from pyspark.sql.types import DoubleType, FloatType

# --- A. Contagem de Nulos por coluna  ---
print("--- Contagem de Valores Nulos --- O objetivo é identificar se há valores nulos ou vazios em campos relevantes")

# Vamos criar uma lista de expressões dinamicamente verificando o tipo da coluna
expressoes_nulos = []

for campo in df_bronze.schema:
    nome_coluna = campo.name
    tipo_dado = campo.dataType
    
    # Se for número , checa nulo OU isNan(Not a number)
    if isinstance(tipo_dado, (DoubleType, FloatType)):
        expressoes_nulos.append(count(when(col(nome_coluna).isNull() | isnan(col(nome_coluna)), nome_coluna)).alias(nome_coluna))
    # Se for texto ou inteiro, checa apenas nulo (isNull) para não dar erro de Cast
    else:
        expressoes_nulos.append(count(when(col(nome_coluna).isNull() , nome_coluna)).alias(nome_coluna))

# Executa a contagem
df_bronze.select(expressoes_nulos).display()


# --- B. Verificar integridade dos campos categóricos (Ex: Gender) ---
print("--- Distribuição de Gênero --- Entender quais os tipos de genero, mas se os ")
# Verifica se a coluna Gender existe antes de tentar agrupar
if "Gender" in df_bronze.columns:
    df_bronze.groupBy("Gender").count().sort(desc("count")).display()


# --- C. Verificar integridade de campos numéricos ---
print("--- Estatísticas Descritivas ---")
df_bronze.describe().display()

### Analise dos dados
Analisando os resultados, não há dados nulos ou não numéricos, ou seja, não precisarei eliminar dados ruins. Com base nos dados, a maioria dos dados são de homens, mas com uma boa representação percentual de mulheres, o que não gera um viés de "opiniões" masculinizadas. Com base nas estatísticas consegue-se entender a média da idade do público, que seria aproximadamente 33 anos, mas com uma variação de 16 a 49 anos. O tempo médio de telas é de 5,52 horas, ou seja, já é um tempo elevado e provavelmente grande parte das pessoas já utiliza demais as telas. Olhando superficialmente, o efeito da média de tempo de horas é refletido na baixa qualidade de sono do grupo. Porém, esse efeito não se repete no índice de felicidade. 


## 3. Etapa
Para essa etapa, identificaram-se os atributos da nova base de dados "Silver". Será feito um filtro simbólico para eliminar dados "nulos" de idade e dados com "tempo de tela" superior a um dia, que na prática seria uma anomalia. 

In [0]:
from pyspark.sql.functions import col, trim, lower, initcap

# 1. Seleção e Renomeação (Mapeamento Bronze -> Silver)
df_silver = df_bronze.select(
    col("User_ID").alias("id_usuario"),
    col("Age").cast("integer").alias("idade"),
    
    # Padronizando Gênero: Remove espaços e coloca primeira letra maiúscula
    initcap(trim(col("Gender"))).alias("genero"),
    
    col("`Daily_Screen_Time(hrs)`").alias("tempo_tela_diario"),
    col("`Sleep_Quality(1-10)`").alias("qualidade_sono"),
    col("`Stress_Level(1-10)`").alias("nivel_estresse"),
    col("Days_Without_Social_Media").alias("dias_sem_redes"),
    col("`Exercise_Frequency(week)`").alias("freq_exercicio"),
    
    # Padronizando Plataforma
    initcap(trim(col("Social_Media_Platform"))).alias("plataforma_rede_social"),
    
    col("`Happiness_Index(1-10)`").alias("indice_felicidade")
)

# 2. Remoção de Duplicatas
df_silver = df_silver.dropDuplicates()

# 3. Filtros de Qualidade 
# Exemplo: Remover linhas onde a idade é nula ou tempo de tela é maior que 24h
df_silver = df_silver.filter(
    (col("idade").isNotNull()) & 
    (col("tempo_tela_diario") <= 24)
)

# Visualizar o resultado limpo
display(df_silver)

## 4. Etapa
Para essa etapa, será criada a base de dados "Silver" que será utilizada para análise de dados e tratar as informações para a base de dados "Gold". 

In [0]:
# Definir o caminho e nome da tabela Silver
# Sugiro manter no mesmo schema 'default' para facilitar
silver_table_name = "workspace.default.silver_social_media_health"

# Salvar como tabela Delta (formato padrão e otimizado do Databricks)
df_silver.write.format("delta").mode("overwrite").saveAsTable(silver_table_name)

print(f"Tabela Silver criada com sucesso: {silver_table_name}")

## 5. Etapa
Para essa etapa, algumas análises de dados foram feitas com o objetivo de identificar valores de correlação e por fim análises com o dataframe original para identificar múltiplas correlações. 

In [0]:
%sql
-- Ver se existe correlação visual rápida entre plataforma e felicidade
SELECT 
  plataforma_rede_social, 
  AVG(indice_felicidade) as media_felicidade,
  AVG(nivel_estresse) as media_estresse,
  AVG(freq_exercicio) as media_exercicio
FROM workspace.default.silver_social_media_health
GROUP BY plataforma_rede_social
ORDER BY media_felicidade DESC

1. Há correlação entre o uso de Redes Sociais e Estresse?

In [0]:

%sql
-- Visão Agrupada (Para Gráfico de Linha ou Dispersão)
SELECT 
  ROUND(tempo_tela_diario, 0) as horas_de_tela,
  ROUND(AVG(nivel_estresse), 2) as media_estresse,
  COUNT(*) as total_pessoas
FROM workspace.default.silver_social_media_health
GROUP BY 1
ORDER BY 1;

-- Cálculo Matemático da Correlação (-1 a 1)
-- Se próximo de 1: Quanto mais tela, mais estresse.
SELECT corr(tempo_tela_diario, nivel_estresse) as correlacao_tela_estresse
FROM workspace.default.silver_social_media_health;

2. Há correlação entre Felicidade e o uso de Redes Sociais?

In [0]:
%sql
SELECT 
  ROUND(tempo_tela_diario, 0) as horas_de_tela,
  ROUND(AVG(indice_felicidade), 2) as media_felicidade
FROM workspace.default.silver_social_media_health
GROUP BY 1
ORDER BY 1;

-- Correlação Estatística
SELECT corr(tempo_tela_diario, indice_felicidade) as correlacao_tela_felicidade
FROM workspace.default.silver_social_media_health;

3. Há correlação entre Redes Sociais e Sono?

In [0]:
%sql
SELECT 
  ROUND(tempo_tela_diario, 0) as horas_de_tela,
  ROUND(AVG(qualidade_sono), 2) as media_sono
FROM workspace.default.silver_social_media_health
GROUP BY 1
ORDER BY 1;

-- Correlação Estatística
SELECT corr(tempo_tela_diario, qualidade_sono) as correlacao_tela_sono
FROM workspace.default.silver_social_media_health;

4. Há correlação entre Sono, Felicidade e Exercícios?

In [0]:
%sql
-- Agrupando por frequência de exercício (Para Gráfico de Barras)
SELECT 
  freq_exercicio,
  ROUND(AVG(qualidade_sono), 2) as media_sono,
  ROUND(AVG(indice_felicidade), 2) as media_felicidade,
  COUNT(*) as total_pessoas
FROM workspace.default.silver_social_media_health
GROUP BY freq_exercicio
ORDER BY freq_exercicio;

5. A idade influência na média de telas, média de sono ou felicidade?

In [0]:
%sql
-- Agrupando por idade de exercício (Para Gráfico de Barras)
SELECT 
  idade,
  ROUND(AVG(tempo_tela_diario), 2) as media_tela,
  ROUND(AVG(qualidade_sono), 0) as media_sono,
  ROUND(AVG(indice_felicidade), 2) as media_felicidade,
  COUNT(*) as total_pessoas
FROM workspace.default.silver_social_media_health
GROUP BY idade
ORDER BY idade;

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

df_analysis = _sqldf.toPandas()

# --- GRÁFICO 1: Idade, Sono e Felicidade em relação à Tela ---
plt.figure()
sns.scatterplot(data=df_analysis, x='idade', y='media_felicidade', 
                hue='media_sono', size='media_tela', sizes=(50, 500), alpha=0.7, palette='deep')
plt.title('Relação Multivariada: Sono vs Felicidade (Tamanho = Exercicio)', fontsize=16)
plt.xlabel('Idade')
plt.ylabel('Média de Felicidade')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='Legenda')
plt.tight_layout()
plt.show()

6. Dados genéricos e distribuições

In [0]:
# Filtro de colunas para fazer o heatmap, correlacionar e plotar
colunas_numericas = ['idade', 'tempo_tela_diario', 'qualidade_sono', 
                     'nivel_estresse', 'indice_felicidade', 'freq_exercicio']

df_pandas = df_silver.select(colunas_numericas).toPandas()
corr_matrix = df_pandas.corr() # O Pandas já calcula tudo e mantém os nomes!

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", vmin=-1, vmax=1)
plt.title('Matriz de Correlação: Hábitos Digitais vs Saúde Mental', fontsize=16)
plt.show()

In [0]:
# 1. Converter para Pandas (Traz os dados para a memória para plotagem)
pdf = df_silver.select(
    'tempo_tela_diario',
    'nivel_estresse',
    'indice_felicidade',
    'qualidade_sono'
).toPandas()

# 2. Configurar layout
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Gráfico 1: Tempo de Tela vs Estresse
# (Este já estava correto usando data=pdf)
sns.regplot(ax=axes[0], data=pdf, x='tempo_tela_diario', y='nivel_estresse', 
            scatter_kws={'alpha':0.3}, line_kws={'color':'red'})
axes[0].set_title('Impacto do Tempo de Tela no Estresse')
axes[0].set_xlabel('Horas de Tela Diárias')
axes[0].set_ylabel('Nível de Estresse (0-10)')

# Gráfico 2: Tempo de Tela vs Felicidade
# CORREÇÃO AQUI: data=pdf
sns.regplot(ax=axes[1], data=pdf, x='tempo_tela_diario', y='indice_felicidade', 
            scatter_kws={'alpha':0.3, 'color':'green'}, line_kws={'color':'blue'})
axes[1].set_title('Impacto do Tempo de Tela na Felicidade')
axes[1].set_xlabel('Horas de Tela Diárias')
axes[1].set_ylabel('Índice de Felicidade')

# Gráfico 3: Tempo de Tela vs Sono
# CORREÇÃO AQUI: data=pdf
sns.regplot(ax=axes[2], data=pdf, x='tempo_tela_diario', y='qualidade_sono', 
            scatter_kws={'alpha':0.3, 'color':'purple'}, line_kws={'color':'black'})
axes[2].set_title('Impacto do Tempo de Tela no Sono')
axes[2].set_xlabel('Horas de Tela Diárias')
axes[2].set_ylabel('Qualidade do Sono (0-10)')

plt.tight_layout()
plt.show()

In [0]:
# Criar um outro dataframe para tratar a distribuição
df_pandas = df_silver.toPandas()

# Configurar layout de grid 2x3 para as distribuições
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 1. Distribuição de Idade
# Note que agora usamos 'df_pandas' em vez de 'df_silver'
sns.histplot(df_pandas['idade'], kde=True, bins=20, ax=axes[0, 0], color='skyblue')
axes[0, 0].set_title('Distribuição por Idade')

# 2. Distribuição de Uso de Tela
sns.histplot(df_pandas['tempo_tela_diario'], kde=True, bins=15, ax=axes[0, 1], color='orange')
axes[0, 1].set_title('Distribuição do Tempo de Tela')

# 3. Distribuição de Frequência de Exercícios
sns.histplot(df_pandas['freq_exercicio'], kde=False, bins=8, ax=axes[0, 2], color='green')
axes[0, 2].set_title('Frequência de Exercícios (Dias/Semana)')

# 4. Distribuição de Estresse
sns.histplot(df_pandas['nivel_estresse'], kde=True, bins=10, ax=axes[1, 0], color='red')
axes[1, 0].set_title('Distribuição dos Níveis de Estresse')

# 5. Distribuição do Sono
sns.histplot(df_pandas['qualidade_sono'], kde=True, bins=10, ax=axes[1, 1], color='purple')
axes[1, 1].set_title('Distribuição da Qualidade do Sono')

# 6. Distribuição da Felicidade
sns.histplot(df_pandas['indice_felicidade'], kde=True, bins=10, ax=axes[1, 2], color='gold')
axes[1, 2].set_title('Distribuição do Índice de Felicidade')

plt.tight_layout()
plt.show()

### Análise dos dados
No primeiro código identificou-se que o Instagram, além de ser a plataforma que tem a média de usuários menos felizes, é a plataforma que possui a média de usuários mais estressados. Já o X, possuí as médias de usuários mais felizes, menos estressados mas que fazem menos exercícios. 

Analisando individualmente a correlação do uso de telas com outras variáveis, percebe-se que há uma certa correlação em relação a felicidade, qualidade do sono e ao estresse. Ou seja, se usar menos telas provavelmente você será mais feliz, menos estressado e dormirá melhor. 

Também analisou-se a relação da frequência dos exercícios e percebe-se que os usuários que fazem mais exercícios possuem maior média de sono e de felicidade, porém, quando analisamos a quantidade total de pessoas, percebe-se que o valor é pouco representativo. Caso fosse interessante treinar um modelo de predição, sugere-se que adicione o poder aquisitivo das pessoas consultadas, se tem filhos e a região na qual elas pertencem, pois imagino que pessoas com maior poder aquisitivo, solteiras e em regiões mais seguras conseguem fazer mais exercícios que em outras condições.

Por fim, analisou-se as correlações, tendências a distribuição. As correlações demonstraram o que as análises anteriores já haviam explicado e a tendência demonstra que em média há uma relação linear, mas aparentemente há um desvio que pode ser gerado por um outro atributo.
 A imagem da distribuição demonstra que aparentemente a distribuição de idade é aleatória, mas o efeito na distribuição de tempo de tela, níveis de stresse e qualidade de sono aparentemente são semelhantes a uma curva normal. A distribuição dos exercícios é semelhante a uma curva logarítmica e a da felicidade é uma crescente. Há algumas lacunas nas distribuições que provavelmente são geradas por conta da baixa quantidade de dados da base, mas por ser tratar de uma análise didática não haverá problemas. Caso os dados sejam utilizados para análises reais deve-se levar em consideração a incerteza da amostra em relação a população, ou seja, os resultados terão uma acurácia atribuida a sua distribuição. Caso alguém siga nesse caminho recomenta-se utilizar os valores da tabela de T-Student para as curvas normais.




## 6. Etapa
Para essa etapa, identificaram-se os atributos da nova base de dados "Silver". Será feito um filtro simbólico para eliminar dados "nulos" de idade e dados com "tempo de tela" superior a um dia, que na prática seria uma anomalia. 

A camada Gold deve conter dados agregados e enriquecidos, prontos para serem consumidos por dashboards (Power BI, Tableau) ou para a apresentação final.

Vamos criar uma tabela Gold que resume o perfil dos usuários, criando "Faixas" (Buckets) para facilitar a análise, em vez de olhar usuário por usuário.

In [0]:
%sql
-- Apagar se já existir para refazer limpo
DROP TABLE IF EXISTS workspace.default.gold_analise_bem_estar;

-- Criar a tabela Gold com dados sumarizados por Perfil
CREATE TABLE workspace.default.gold_analise_bem_estar AS
SELECT
  genero,
  plataforma_rede_social,
  
  -- Criando uma categoria de uso para facilitar filtros no Dashboard
  CASE 
    WHEN tempo_tela_diario <= 2 THEN '1. Baixo Uso (0-2h)'
    WHEN tempo_tela_diario <= 5 THEN '2. Uso Moderado (2-5h)'
    WHEN tempo_tela_diario <= 8 THEN '3. Alto Uso (5-8h)'
    ELSE '4. Uso Extremo (>8h)'
  END AS categoria_uso_tela,
  
  -- Métricas Médias (KPIs)
  ROUND(AVG(nivel_estresse), 2) as media_estresse,
  ROUND(AVG(indice_felicidade), 2) as media_felicidade,
  ROUND(AVG(qualidade_sono), 2) as media_sono,
  ROUND(AVG(freq_exercicio), 2) as media_exercicio,

  -- Contagem para saber a relevância estatística
  COUNT(*) as total_usuarios

FROM workspace.default.silver_social_media_health
GROUP BY 
  genero, 
  plataforma_rede_social, 
  categoria_uso_tela
ORDER BY 
  categoria_uso_tela, 
  plataforma_rede_social;

Após criar a Gold, faça um select simples para ver como ficou sua tabela final:

In [0]:
%sql
SELECT * FROM workspace.default.gold_analise_bem_estar

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 1. Carregar os dados da tabela Gold para o Pandas
# O 'toPandas()' traz os dados para a memória do driver para plotagem
df_gold = spark.read.table("workspace.default.gold_analise_bem_estar").toPandas()

# Configurações visuais gerais
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (14, 7) # Aumenta o tamanho padrão para melhor leitura

# --- GRÁFICO 1: Gênero e Média de Stress por Categoria de Uso de Tela ---
plt.figure()
sns.barplot(data=df_gold, x='categoria_uso_tela', y='media_estresse', hue='genero', palette='viridis')
plt.title('Média de Estresse por Categoria de Uso de Tela e Gênero', fontsize=16)
plt.xlabel('Categoria de Uso de Tela')
plt.ylabel('Média de Estresse')
plt.xticks(rotation=45)
plt.legend(title='Gênero')
plt.tight_layout()
plt.show() # No Databricks, use display() se preferir ou plt.show()

# --- GRÁFICO 2: Gênero e Stress por Plataforma ---
plt.figure()
sns.barplot(data=df_gold, x='plataforma_rede_social', y='media_estresse', hue='genero', palette='magma')
plt.title('Média de Estresse por Plataforma de Rede Social e Gênero', fontsize=16)
plt.xlabel('Plataforma')
plt.ylabel('Média de Estresse')
plt.legend(title='Gênero')
plt.tight_layout()
plt.show()

# --- GRÁFICO 3: Sono e Categoria de Uso de Tela ---
plt.figure()
# Agrupamos para garantir que temos uma média única por categoria caso haja subdivisões
df_sono = df_gold.groupby('categoria_uso_tela')['media_sono'].mean().reset_index()
sns.barplot(data=df_sono, x='categoria_uso_tela', y='media_sono', palette='Blues_d')
plt.title('Média de Qualidade do Sono por Categoria de Uso de Tela', fontsize=16)
plt.xlabel('Categoria de Uso de Tela')
plt.ylabel('Qualidade do Sono (Média)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# --- GRÁFICO 4: Categoria Uso Tela x Stress x Felicidade ---
plt.figure()
# Transformamos os dados para formato "longo" para o Seaborn plotar barras agrupadas das métricas
df_melted = df_gold.melt(id_vars=['categoria_uso_tela'], value_vars=['media_estresse', 'media_felicidade'], 
                         var_name='Métrica', value_name='Valor')
sns.barplot(data=df_melted, x='categoria_uso_tela', y='Valor', hue='Métrica', palette='coolwarm')
plt.title('Comparativo: Estresse vs Felicidade por Categoria de Uso', fontsize=16)
plt.xlabel('Categoria de Uso de Tela')
plt.ylabel('Valor Médio (Escala 0-10)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# --- GRÁFICO 5: Stress, Sono e Gênero em relação à Felicidade ---
plt.figure()
sns.scatterplot(data=df_gold, x='media_estresse', y='media_felicidade', 
                hue='genero', size='media_sono', sizes=(50, 500), alpha=0.7, palette='deep')
plt.title('Relação Multivariada: Estresse vs Felicidade (Tamanho = Qualidade do Sono)', fontsize=16)
plt.xlabel('Média de Estresse')
plt.ylabel('Média de Felicidade')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='Legenda')
plt.tight_layout()
plt.show()

# --- GRÁFICO 6: Rede Social com Uso de Tela e Felicidade ---
plt.figure()
sns.barplot(data=df_gold, x='plataforma_rede_social', y='media_felicidade', hue='categoria_uso_tela', palette='Set2')
plt.title('Felicidade por Plataforma e Intensidade de Uso', fontsize=16)
plt.xlabel('Plataforma')
plt.ylabel('Média de Felicidade')
plt.legend(title='Uso de Tela', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# --- GRÁFICO 7: Stress, Exercicio e Gênero em relação à Felicidade ---
plt.figure()
sns.scatterplot(data=df_gold, x='media_estresse', y='media_felicidade', 
                hue='genero', size='media_exercicio', sizes=(50, 500), alpha=0.7, palette='deep')
plt.title('Relação Multivariada: Estresse vs Felicidade (Tamanho = Exercicio)', fontsize=16)
plt.xlabel('Média de Estresse')
plt.ylabel('Média de Felicidade')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='Legenda')
plt.tight_layout()
plt.show()

## Respostas para as perguntas de Negócio 

1.  **Redes Sociais x Estresse:** Existe uma correlação entre o tempo gasto em redes sociais e o aumento do nível de estresse? 

  **Resposta:** Conforme o item 5.1, identificou-se uma correlação de aproximadamente 0.74 entre redes sociais e o aumento do nível de estresse.
  
   Conforme o gráfico "Média de Estresse por Categoria de Uso de Tela e Gênero" identificou-se que indepentende do gênero, em média quanto mais tempo você usa telas, maior o seu nível de estresse.

  Já com o gráfico "Média de Estresse por Plataforma de Rede Social e Gênero" identificamos que plataforma que negativamente se destaca em média é o "Instagram", independênte do gênero. 

  Esse resultado intrigou o desenvolvedor desse trabalho pois, por ser um usuário do Instagram, entendeu-se que precisa reduzir/eliminar o acesso à essa plataforma, mas fica a reflexão:
  
  O uso é algo que gera estresse ou o uso é uma fuga para problemas e estes são os geradores de estresse? Será que esse resultado é um reflexo de uma doença silenciosa que está dominando a sociedade? Para responder isso, o desenvolvedor entende que são necessários mais dados e atributos, como: poder aquisitivo, região, formação, emprego, estado civil, se está estudando, entre outros. 

2.  **Redes Sociais x Felicidade:** O uso intensivo de plataformas digitais afeta o índice de felicidade reportado?

**Resposta:** Conforme o item 5.2 há uma correlação de -0.70, ou seja, em média quando maior o uso de telas, menos a felicidade. 

Quando analisamos o gráfico "Comparativo: Estresse vs Felicidade por Categoria de Uso" percebe-se que em média as pessoas que usam menos telas são mais felizes e menos estressadas. 

Fica a dúvida se há uma causa raiz, como por exemplo, se a pessoa usa mais telas ela fica mais estressada e por conseguinte ela fica menos feliz? Ou se as pessoas que usam mais telas são menos felizes e isso gera um estresse maior. Independente da resposta, fica claro que o uso excessivo de telas não faz bem, mas entender o agente causador auxilia na resolução do problema. No entendimento do desenvolvedor faltam dados para saber qual o causador, pois a pessoa pode estar infeliz por ter perdido um parente ou ela pode estar estressada por conta de demandas no trabalho. 

3.  **Redes Sociais x Sono:** O tempo de tela diário impacta a qualidade do sono?

**Resposta:** Conforme o item 5.3 há uma correlação entre o tempo médio de uso de telas e a qualidade do sono de -0.76, ou seja, quanto maior o tempo de telas menor a qualidade do sono. 


4.  **Hábitos Saudáveis:** Qual a relação entre a frequência de exercícios físicos, qualidade do sono e a felicidade geral?

**Resposta:** Conforme o item 5.4 avaliando a tabela percebe-se que em média não há uma relação direta entre o aumento da frequência de exercícios físicos e a qualidade do sono e a felicidade. Se analisarmos apenas os maiores valores de frequência de exercícios físicos, percebe-se que neles se concentram os maiores valores de qualidade de sono e de felicidade, porém, o argumento do desenvolvedor se sustentam em duas hipóteses:

- A amostragem de pessoas que praticam uma elevada frequência de exercícios físicos é baixa, logo imagina-se que são pessoas que possuem alto poder aquisitivo e por conseguinte podem ter outros fatores que influenciam tanto quanto os exercicíos. 

- Pessoas que não praticam exercícios são mais felizes e possuem maior média de sono que as pessoas que praticam duas vezes. 


5.  **Outras conclusões:** 

Complementando essa análise, cita-se o gráfico 
"Relação Multivariada: Estresse vs Felicidade (Tamanho = Qualidade do Sono)", nele percebe-se que há uma relação de que quando maior o estresse, menor a felicidade e pior a qualidade do sono. Já quando avalia-se o gráfico "Relação Multivariada: Estresse vs Felicidade (Tamanho = Exercicio)" que demonstra que não há uma relação direta do exercício, estresse e felicidade. 

Já no gráfico "Felicidade por Plataforma e Intensidade de Uso", percebe-se que independente da plataforma, em média quanto maior o uso, mais prejudicial é para a felicidade. 

Com os dados do item 5.5, tentou-se avaliar se alguma geração(exemplo: X, Y ou Z) possuia algum destaque em relação a felicidade ou se alguma faixa de idade que pode ser casada teria alguma relação com o uso de telas, mas não dá para afirmar isso pois há algumas variações que parecem ser aleatórias. Imagina-se que isso ocorre pois faltam atributos para classificarmos melhor os grupos. 

6.  **Conclusão final:** Entende-se que o uso em excesso de telas poderá gerar estresse e reduzir a felicidade. Esse problema não escolhe gênero nem idade, logo estamos falando de uma pandemia silenciosa que prejudica cada vez mais a saúde mental das pessoas. Evite usa-las.  

